In [48]:
# Peter Karras 2020
'''Takes GPP and date information from old master file and creates a new Machine Learning Master File
   With one column representing file names for the converted CSVs and the other column being their respective
   GPPs as pulled from the old master file or calculated with linear interpolation'''

'Takes GPP and date information from old master file and creates a new Machine Learning Master File\n   With one column representing file names for the converted CSVs and the other column being their respective\n   GPPs as pulled from the old master file or calculated with linear interpolation'

In [49]:
import pandas as pd
import os

In [50]:
# Takes in a string that represents name of csv file and 
# converts that string to one that matches the date string present
# in the old master file (old master file refers to the raw excel file
# filled with extraneous data from BCI)that contains the date/times 
# for recorded GPP values
def change_csv_date(csv):
    year = csv[2:4]
    month = csv[5:7]
    day = csv[8:10]
    hour = csv[11:13]
    minute = csv[14:16]
    second = "00"
    if(int(hour) == 0):
        hour = "0"
    elif(int(hour) < 10):
        hour = hour[1:2]
    # No leading zeroes in day or month
    if(int(month) < 10):
        month = month[1:2]
    if(int(day) < 10):
        day = day[1:2]
    result = month + "/" + day + "/" + year + " " + hour + ":" + minute #+ ":" + second + " " + timeDay
    return result

In [85]:
# Grabs all GPP values and dates from master file (as defined above)
masterfileLocation = r"C:\Users\Peter\Desktop\Thermo_R_Code\FinalThermo"
os.chdir(masterfileLocation)
col_list = ["date", "GPP"]
masterFileName = "all-30min-Apr2018_working.csv"
dateGPPList = pd.read_csv(masterFileName, usecols=col_list) 
dateGPPList = dateGPPList.set_index("date")

# This looks at local files using getcwd (get current working directory) 
# and will look into a folder named "csvdata" (no quotes) and grab the file 
# names of all the files that have been made into CSV's from the original
# .seq files using the R conversion program in that "csvdata" folder 

# csvdataLocation = r"C:\Users\Peter\Desktop\Thermo_R_Code\FinalThermo\csvdata"
# os.chdir(csvdataLocation)
# csvfiles = [os.path.abspath(x) for x in os.listdir()]

csvdataLocation = r"C:\Users\Peter\Desktop\Thermo_R_Code\FinalThermo\csvdata"
(_, _, csvfiles) = next(os.walk(csvdataLocation))
print(csvfiles[0])

2015_03_14_13_30_bci_t.csv


In [86]:
# This chunk of code matches the file names from the old master file to the
# names gotten from the "csvdata" folder, so that each thermal csv file has it's
# corresponding GPP value matched to it
# NOTE: In addition to matching GPP values, this code also fills in GPP values
# for those images without recorded values through linear interpoaltion
# NOTE: Currently only works when first CSV is from a 30 min increment
dictList = list()

counter = 0
numLoops = len(csvfiles)
oldGPP = -1.1
for i in range(0, numLoops, 6):
    #GPPDataDict.clear()
    currFile = csvfiles[i]
    adaptedDate = change_csv_date(currFile)
    print(adaptedDate)
    currGPP = dateGPPList.at[adaptedDate,"GPP"]
    if(i == 0):
        #GPPDataDict.update({currFile : currGPP})
        GPPDataDict = dict()
        GPPDataDict.update({"GPP" : currGPP})
        GPPDataDict.update({"FILE" : currFile})
        dictList.append(GPPDataDict)
        oldGPP = currGPP
        continue
        
    GPPDataDictTemp = dict()
    GPPDataDictTemp.update({"GPP" : currGPP})
    GPPDataDictTemp.update({"FILE" : currFile})
    
    
    diffGPP = currGPP - oldGPP
    increment = diffGPP / 6.0
    for j in range(i-5, i, 1):
        oldGPP += increment
        currFile = csvfiles[j]
        GPPDataDict = dict()
        GPPDataDict.update({"GPP" : oldGPP})
        GPPDataDict.update({"FILE" : currFile})
        dictList.append(GPPDataDict)
    dictList.append(GPPDataDictTemp)
    oldGPP = currGPP + increment
    oldGPP = currGPP

#print(dictList)

3/14/15 13:30
3/14/15 14:00
3/14/15 14:30
3/14/15 15:00
3/14/15 15:30
3/14/15 16:00
3/14/15 16:30
3/14/15 17:00
3/14/15 17:30
3/14/15 18:00
3/14/15 18:30
3/14/15 19:00
3/14/15 19:30
3/14/15 20:00
3/14/15 20:30
3/14/15 21:00
3/14/15 21:30
3/14/15 22:00
3/14/15 22:30
3/14/15 23:00
3/14/15 23:30
3/15/15 0:00
3/15/15 0:30
3/15/15 1:00
3/15/15 1:30
3/15/15 2:00
3/15/15 2:30
3/15/15 3:00
3/15/15 3:30
3/15/15 4:00
3/15/15 4:30
3/15/15 5:00
3/15/15 5:30
3/15/15 6:00
3/15/15 6:30
3/15/15 7:00
3/15/15 7:30
3/15/15 8:00
3/15/15 8:30
3/15/15 9:00
3/15/15 9:30
3/15/15 10:00
3/15/15 10:30
3/15/15 11:00
3/15/15 11:30
3/15/15 12:00
3/15/15 12:30
3/15/15 13:00
3/15/15 13:30
3/15/15 14:00
3/15/15 14:30
3/15/15 15:00
3/15/15 15:30
3/15/15 16:00
3/15/15 16:30
3/15/15 17:00
3/15/15 17:30
3/15/15 18:00
3/15/15 18:30
3/15/15 19:00
3/15/15 19:30
3/15/15 20:00
3/15/15 20:30
3/15/15 21:00
3/15/15 21:30
3/15/15 22:00
3/15/15 22:30
3/15/15 23:00
3/15/15 23:30
3/16/15 0:00
3/16/15 0:30
3/16/15 1:00
3/16/15 1:30
3/

In [87]:
# This writes a NEW master file, with one column containing the
# thermal csv file names, and the other column containing their
# GPP values as determined from previous code. New file name is 
# determined by variable csv_file1

import csv
#saveLocation = ""
#os.chdir(saveLocation)
csv_file1 = "populatedGPPs.csv"
csv_columns = ['FILE','GPP']
try:
    with open(csv_file1, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, delimiter=',', lineterminator='\n', fieldnames=csv_columns)
        writer.writeheader()
        for data in dictList:
            writer.writerow(data)
except IOError:
    print("I/O error")
    
